In [7]:
from newsapi.newsapi_client import NewsApiClient
import pandas as pd
from newspaper import Article, Config
import nltk
from nltk.corpus import stopwords
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from datetime import date, timedelta
today = date.today()
yesterday = today - timedelta(days=1)
import shutup; shutup.please()


# Init
newsapi = NewsApiClient(api_key='ff4373852c2343a98303951439854f8c')

# /v2/top-headlines
top_headlines = newsapi.get_top_headlines(
                                          category='general',
                                          language='en',
                                          page_size=90,
                                          page=1
                                        )

# source = https://newsapi.org/docs/client-libraries/python
articles = top_headlines.get('articles', [])

    # Create a DataFrame with specific columns
initial_df = pd.DataFrame(articles, columns=['source','title','publishedAt','author','url'])

# Data Cleaning
initial_df['source'] = initial_df['source'].apply(lambda x: x['name'] if pd.notna(x) and 'name' in x else None)

initial_df['publishedAt'] = pd.to_datetime(initial_df['publishedAt'])

yesterday_data = initial_df[initial_df['publishedAt'].dt.date == yesterday]


df = yesterday_data.copy()
df

,source,title,publishedAt,author,url
0,CBS Sports,"2024 NCAA Tournament scores, schedule: March M...",2024-03-24 20:12:02+00:00,,https://www.cbssports.com/college-basketball/n...
1,CNN,New Jersey first lady Tammy Murphy is suspendi...,2024-03-24 20:03:00+00:00,Gregory Krieg,https://www.cnn.com/2024/03/24/politics/tammy-...
2,The Hill,‘Severe’ geomagnetic storm conditions impactin...,2024-03-24 19:50:17+00:00,Addy Bink,https://thehill.com/homenews/nexstar_media_wir...
4,TMZ,'Yellowstone' Star Says He Was Kicked Off Plan...,2024-03-24 19:40:00+00:00,TMZ Staff,https://www.tmz.com/2024/03/24/yellowstone-sta...
5,ESPN,"Reigan Richardson, Duke rally past Ohio State,...",2024-03-24 19:39:00+00:00,None,https://www.espn.com/womens-college-basketball...
...,...,...,...,...,...
71,India.com,iPhone 14 Plus Available With Big Discounts On...,2024-03-24 04:04:24+00:00,Pawan Rai,https://zeenews.india.com/technology/iphone-14...
72,NDTV News,Astrophysicist Claims He Has Cracked Equation ...,2024-03-24 03:45:07+00:00,None,https://www.ndtv.com/science/astrophysicist-cl...
73,GameSpot,Dragon's Dogma 2 - Magick-Archer Vocation And ...,2024-03-24 03:32:59+00:00,Jason Rodriguez,https://www.gamespot.com/articles/dragons-dogm...
74,Detroit Free Press,"Powerball numbers for Saturday, March 23, 2024...",2024-03-24 03:01:50+00:00,Tanya Wildt,https://www.freep.com/story/news/local/michiga...


## Getting full articles

In [8]:
from newspaper import Article, Config

def full_content(url):
    user_agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'
    config = Config()
    config.browser_user_agent = user_agent
    page = Article(url, config=config)
    
    try:
        page.download()
        page.parse()
        return page.text
    except Exception as e:
        print(f"Error retrieving content from {url}: {e}")
        return 'couldnt retrieve'




df['content'] = df['url'].apply(full_content)
df['content'] = df['content'].str.replace('\n', ' ')

df = df[df['content'] != 'couldnt retrieve']
df


Error retrieving content from https://www.usatoday.com/story/news/nation/2024/03/24/mountain-lion-fatal-attack-california/73085116007/: Article `download()` failed with 403 Client Error: Unknown Error for url: https://eu.usatoday.com/story/news/nation/2024/03/24/mountain-lion-fatal-attack-california/73085116007/ on URL https://www.usatoday.com/story/news/nation/2024/03/24/mountain-lion-fatal-attack-california/73085116007/
Error retrieving content from https://www.newsobserver.com/sports/college/acc/nc-state/article287036145.html: Article `download()` failed with HTTPSConnectionPool(host='www.newsobserver.com', port=443): Read timed out. (read timeout=7) on URL https://www.newsobserver.com/sports/college/acc/nc-state/article287036145.html
Error retrieving content from https://www.usatoday.com/story/news/politics/2024/03/24/aoc-mike-johnson-democrats-support/73084686007/: Article `download()` failed with 403 Client Error: Unknown Error for url: https://eu.usatoday.com/story/news/politics

,source,title,publishedAt,author,url,content
0,CBS Sports,"2024 NCAA Tournament scores, schedule: March M...",2024-03-24 20:12:02+00:00,,https://www.cbssports.com/college-basketball/n...,The 2024 NCAA Tournament continues Thursday wi...
1,CNN,New Jersey first lady Tammy Murphy is suspendi...,2024-03-24 20:03:00+00:00,Gregory Krieg,https://www.cnn.com/2024/03/24/politics/tammy-...,CNN — New Jersey first lady Tammy Murphy is s...
2,The Hill,‘Severe’ geomagnetic storm conditions impactin...,2024-03-24 19:50:17+00:00,Addy Bink,https://thehill.com/homenews/nexstar_media_wir...,(NEXSTAR) — A major geomagnetic storm is impac...
4,TMZ,'Yellowstone' Star Says He Was Kicked Off Plan...,2024-03-24 19:40:00+00:00,TMZ Staff,https://www.tmz.com/2024/03/24/yellowstone-sta...,Play video content Forrie J. Smith claims he ...
5,ESPN,"Reigan Richardson, Duke rally past Ohio State,...",2024-03-24 19:39:00+00:00,None,https://www.espn.com/womens-college-basketball...,Reigan Richardson and Duke claw back from a do...
...,...,...,...,...,...,...
71,India.com,iPhone 14 Plus Available With Big Discounts On...,2024-03-24 04:04:24+00:00,Pawan Rai,https://zeenews.india.com/technology/iphone-14...,"New Delhi: Flipkart, a popular e-commerce plat..."
72,NDTV News,Astrophysicist Claims He Has Cracked Equation ...,2024-03-24 03:45:07+00:00,None,https://www.ndtv.com/science/astrophysicist-cl...,The professor spent his career investigating t...
73,GameSpot,Dragon's Dogma 2 - Magick-Archer Vocation And ...,2024-03-24 03:32:59+00:00,Jason Rodriguez,https://www.gamespot.com/articles/dragons-dogm...,You can choose from various vocations or class...
74,Detroit Free Press,"Powerball numbers for Saturday, March 23, 2024...",2024-03-24 03:01:50+00:00,Tanya Wildt,https://www.freep.com/story/news/local/michiga...,couldnt retrieve


,source,title,publishedAt,author,url,content
0,CBS Sports,"2024 NCAA Tournament scores, schedule: March M...",2024-03-24 20:12:02+00:00,,https://www.cbssports.com/college-basketball/n...,The 2024 NCAA Tournament continues Thursday wi...
1,CNN,New Jersey first lady Tammy Murphy is suspendi...,2024-03-24 20:03:00+00:00,Gregory Krieg,https://www.cnn.com/2024/03/24/politics/tammy-...,CNN — New Jersey first lady Tammy Murphy is s...
2,The Hill,‘Severe’ geomagnetic storm conditions impactin...,2024-03-24 19:50:17+00:00,Addy Bink,https://thehill.com/homenews/nexstar_media_wir...,(NEXSTAR) — A major geomagnetic storm is impac...
4,TMZ,'Yellowstone' Star Says He Was Kicked Off Plan...,2024-03-24 19:40:00+00:00,TMZ Staff,https://www.tmz.com/2024/03/24/yellowstone-sta...,Play video content Forrie J. Smith claims he ...
5,ESPN,"Reigan Richardson, Duke rally past Ohio State,...",2024-03-24 19:39:00+00:00,None,https://www.espn.com/womens-college-basketball...,Reigan Richardson and Duke claw back from a do...
...,...,...,...,...,...,...
70,The Times of India,"T+0 settlement, Nifty rejig, F&O expiry among ...",2024-03-24 04:15:40+00:00,Vidya Sreedhar,https://economictimes.indiatimes.com/markets/s...,T+0 Settlement F&O Expiry Broader Market Act...
71,India.com,iPhone 14 Plus Available With Big Discounts On...,2024-03-24 04:04:24+00:00,Pawan Rai,https://zeenews.india.com/technology/iphone-14...,"New Delhi: Flipkart, a popular e-commerce plat..."
72,NDTV News,Astrophysicist Claims He Has Cracked Equation ...,2024-03-24 03:45:07+00:00,None,https://www.ndtv.com/science/astrophysicist-cl...,The professor spent his career investigating t...
73,GameSpot,Dragon's Dogma 2 - Magick-Archer Vocation And ...,2024-03-24 03:32:59+00:00,Jason Rodriguez,https://www.gamespot.com/articles/dragons-dogm...,You can choose from various vocations or class...


## Word Count without stopwords

In [10]:
import pandas as pd
import nltk
from nltk.corpus import stopwords

# Download the stopwords dataset
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

# Function to count words without stopwords
def count_words_without_stopwords(text):
    if isinstance(text, (str, bytes)):
        words = nltk.word_tokenize(str(text))
        stop_words = set(stopwords.words('english'))
        filtered_words = [word for word in words if word.lower() not in stop_words]
        return len(filtered_words)
    else:
        return 0

# Apply the function to the 'content' column using .loc for assignment
df['WordCount'] = df['content'].apply(count_words_without_stopwords)
df

[nltk_data] Downloading package stopwords to /home/gitpod/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/gitpod/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/gitpod/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


,source,title,publishedAt,author,url,content,WordCount
0,CBS Sports,"2024 NCAA Tournament scores, schedule: March M...",2024-03-24 20:12:02+00:00,,https://www.cbssports.com/college-basketball/n...,The 2024 NCAA Tournament continues Thursday wi...,702
1,CNN,New Jersey first lady Tammy Murphy is suspendi...,2024-03-24 20:03:00+00:00,Gregory Krieg,https://www.cnn.com/2024/03/24/politics/tammy-...,CNN — New Jersey first lady Tammy Murphy is s...,507
2,The Hill,‘Severe’ geomagnetic storm conditions impactin...,2024-03-24 19:50:17+00:00,Addy Bink,https://thehill.com/homenews/nexstar_media_wir...,(NEXSTAR) — A major geomagnetic storm is impac...,723
4,TMZ,'Yellowstone' Star Says He Was Kicked Off Plan...,2024-03-24 19:40:00+00:00,TMZ Staff,https://www.tmz.com/2024/03/24/yellowstone-sta...,Play video content Forrie J. Smith claims he ...,226
5,ESPN,"Reigan Richardson, Duke rally past Ohio State,...",2024-03-24 19:39:00+00:00,None,https://www.espn.com/womens-college-basketball...,Reigan Richardson and Duke claw back from a do...,328
...,...,...,...,...,...,...,...
70,The Times of India,"T+0 settlement, Nifty rejig, F&O expiry among ...",2024-03-24 04:15:40+00:00,Vidya Sreedhar,https://economictimes.indiatimes.com/markets/s...,T+0 Settlement F&O Expiry Broader Market Act...,529
71,India.com,iPhone 14 Plus Available With Big Discounts On...,2024-03-24 04:04:24+00:00,Pawan Rai,https://zeenews.india.com/technology/iphone-14...,"New Delhi: Flipkart, a popular e-commerce plat...",266
72,NDTV News,Astrophysicist Claims He Has Cracked Equation ...,2024-03-24 03:45:07+00:00,None,https://www.ndtv.com/science/astrophysicist-cl...,The professor spent his career investigating t...,288
73,GameSpot,Dragon's Dogma 2 - Magick-Archer Vocation And ...,2024-03-24 03:32:59+00:00,Jason Rodriguez,https://www.gamespot.com/articles/dragons-dogm...,You can choose from various vocations or class...,705


In [11]:
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

# Download the VADER lexicon
nltk.download('vader_lexicon')

# Create a SentimentIntensityAnalyzer object
sid = SentimentIntensityAnalyzer()


# Function to get sentiment scores and label
def get_sentiment(row):
    sentiment_scores = sid.polarity_scores(row)
    compound_score = sentiment_scores['compound']

    if compound_score >= 0.05:
        sentiment = 'Positive'
    elif compound_score <= -0.05:
        sentiment = 'Negative'
    else:
        sentiment = 'Neutral'

    return sentiment, compound_score

# Apply sentiment analysis to each row of the DataFrame
df[['Sentiment', 'Compound_Score']] = df['content'].astype(str).apply(lambda x: pd.Series(get_sentiment(x)))
df



[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/gitpod/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


,source,title,publishedAt,author,url,content,WordCount,Sentiment,Compound_Score
0,CBS Sports,"2024 NCAA Tournament scores, schedule: March M...",2024-03-24 20:12:02+00:00,,https://www.cbssports.com/college-basketball/n...,The 2024 NCAA Tournament continues Thursday wi...,702,Positive,0.9689
1,CNN,New Jersey first lady Tammy Murphy is suspendi...,2024-03-24 20:03:00+00:00,Gregory Krieg,https://www.cnn.com/2024/03/24/politics/tammy-...,CNN — New Jersey first lady Tammy Murphy is s...,507,Positive,0.9904
2,The Hill,‘Severe’ geomagnetic storm conditions impactin...,2024-03-24 19:50:17+00:00,Addy Bink,https://thehill.com/homenews/nexstar_media_wir...,(NEXSTAR) — A major geomagnetic storm is impac...,723,Positive,0.9339
4,TMZ,'Yellowstone' Star Says He Was Kicked Off Plan...,2024-03-24 19:40:00+00:00,TMZ Staff,https://www.tmz.com/2024/03/24/yellowstone-sta...,Play video content Forrie J. Smith claims he ...,226,Positive,0.8755
5,ESPN,"Reigan Richardson, Duke rally past Ohio State,...",2024-03-24 19:39:00+00:00,None,https://www.espn.com/womens-college-basketball...,Reigan Richardson and Duke claw back from a do...,328,Positive,0.9298
...,...,...,...,...,...,...,...,...,...
70,The Times of India,"T+0 settlement, Nifty rejig, F&O expiry among ...",2024-03-24 04:15:40+00:00,Vidya Sreedhar,https://economictimes.indiatimes.com/markets/s...,T+0 Settlement F&O Expiry Broader Market Act...,529,Positive,0.9915
71,India.com,iPhone 14 Plus Available With Big Discounts On...,2024-03-24 04:04:24+00:00,Pawan Rai,https://zeenews.india.com/technology/iphone-14...,"New Delhi: Flipkart, a popular e-commerce plat...",266,Positive,0.9804
72,NDTV News,Astrophysicist Claims He Has Cracked Equation ...,2024-03-24 03:45:07+00:00,None,https://www.ndtv.com/science/astrophysicist-cl...,The professor spent his career investigating t...,288,Negative,-0.2130
73,GameSpot,Dragon's Dogma 2 - Magick-Archer Vocation And ...,2024-03-24 03:32:59+00:00,Jason Rodriguez,https://www.gamespot.com/articles/dragons-dogm...,You can choose from various vocations or class...,705,Negative,-0.8782


In [16]:

import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
from io import BytesIO
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient
from dotenv import load_dotenv
import os
from datetime import date, timedelta
today = date.today()
yesterday = today - timedelta(days=1)



load_dotenv()

# Convert DataFrame to Arrow Table
table = pa.Table.from_pandas(df)

parquet_buffer = BytesIO()
pq.write_table(table, parquet_buffer)

connection_string = os.getenv('BLOB_CONN_STRING')
blob_service_client = BlobServiceClient.from_connection_string(connection_string)

container_name = "testtech"
blob_name = f"news-{yesterday}.parquet"
container_client = blob_service_client.get_container_client(container_name)

blob_client = container_client.get_blob_client(blob_name)
blob_client.upload_blob(parquet_buffer.getvalue(), overwrite=True)
print(f"{blob_name} successfully updated")


news-2024-03-24.parquet successfully updated


## top keywords

In [14]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist
import string

nltk.download('stopwords')
nltk.download('punkt')

def preprocess_text(text):
    # Tokenize the text
    tokens = word_tokenize(text)
    
    # Convert to lowercase
    tokens = [word.lower() for word in tokens]
    
    # Remove punctuation
    tokens = [word for word in tokens if word.isalnum()]
    
    # Remove stop words
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]

    return tokens

def get_top_keywords(tokens, top_n=30):
    # Calculate word frequencies
    freq_dist = FreqDist(tokens)
    
    # Get the top N keywords
    top_keywords = freq_dist.most_common(top_n)
    
    return top_keywords

text_data_column = df['content']

# Convert the column values to a single text variable
text_data = ' '.join(text_data_column.astype(str).tolist())

# Preprocess the text
preprocessed_tokens = preprocess_text(text_data)

# Get the top keywords
top_keywords = get_top_keywords(preprocessed_tokens)

# Print or use the top keywords
top_keywords

[nltk_data] Downloading package stopwords to /home/gitpod/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/gitpod/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


[('said', 159),
 ('also', 76),
 ('india', 64),
 ('people', 57),
 ('new', 48),
 ('court', 48),
 ('apple', 48),
 ('way', 46),
 ('two', 45),
 ('israel', 42),
 ('mattress', 42),
 ('one', 40),
 ('world', 39),
 ('bajaj', 39),
 ('news', 38),
 ('galaxy', 38),
 ('first', 36),
 ('space', 35),
 ('stars', 35),
 ('company', 34),
 ('last', 33),
 ('bhutan', 33),
 ('friday', 32),
 ('gaza', 32),
 ('million', 31),
 ('would', 31),
 ('day', 30),
 ('us', 30),
 ('years', 29),
 ('hamas', 29)]

In [15]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt


# Convert the list of tuples to a dictionary for WordCloud input
wordcloud_dict = dict(top_keywords)

# Create a WordCloud object
wordcloud = WordCloud(width=800, height=400, background_color='black').generate_from_frequencies(wordcloud_dict)

# Display the WordCloud using matplotlib
plt.figure(figsize=(10, 5))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')  # Turn off the axis labels
plt.show()

ModuleNotFoundError: No module named 'wordcloud'

## Topic modelling

In [ ]:
import gensim
from gensim import corpora
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import string
import pyLDAvis.gensim_models as gensimvis
import pyLDAvis

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
# text data from df column to list
text_data = df['content'].tolist()



def preprocess_text_list(text_list, remove_words=None, remove_single_alphabet=True):
    preprocessed_texts = []

    for text in text_list:
        # Tokenize the text
        tokens = word_tokenize(text.lower())

        # Remove stopwords, punctuation, and digits
        stop_words = set(stopwords.words('english'))
        tokens = [token for token in tokens if token not in stop_words and token not in string.punctuation and not token.isdigit()]

        # Remove specific words
        if remove_words:
            tokens = [token for token in tokens if token not in remove_words]

        # Remove 'n't' and other contractions
        tokens = [token for token in tokens if "’" not in token and "\'\'" not in token]

        # Remove singular alphabet characters
        if remove_single_alphabet:
            tokens = [token for token in tokens if len(token) > 1 or token.lower() == 'a' or token.lower() == 'i']

        # Lemmatize words
        lemmatizer = WordNetLemmatizer()
        tokens = [lemmatizer.lemmatize(token) for token in tokens]

        preprocessed_texts.append(tokens)

    return preprocessed_texts

preprocessed_texts_list = preprocess_text_list(text_data, remove_words=['``',"\'s"], remove_single_alphabet=True)

# Preprocess each document in the text data
#preprocessed_data = [preprocess_text(doc) for doc in text_data]

# Create a dictionary and a corpus
dictionary = corpora.Dictionary(preprocessed_texts_list)
corpus = [dictionary.doc2bow(doc) for doc in preprocessed_texts_list]

# Build the LDA model
lda_model = gensim.models.LdaModel(corpus, num_topics=5, id2word=dictionary, passes=15)
# Visualize the LDA model
lda_vis_data = gensimvis.prepare(lda_model, corpus, dictionary)
# pyLDAvis.display(lda_vis_data)
lda_model.print_topics()

[nltk_data] Downloading package punkt to /home/gitpod/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/gitpod/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/gitpod/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
/workspace/.pyenv_mirror/user/current/lib/python3.12/site-packages/joblib/_utils.py:38: DeprecationWarning: ast.Num is deprecated and will be removed in Python 3.14; use ast.Constant instead
  if isinstance(node, ast.Num):  # <number>
/workspace/.pyenv_mirror/user/current/lib/python3.12/site-packages/joblib/_utils.py:38: DeprecationWarning: ast.Num is deprecated and will be removed in Python 3.14; use ast.Constant instead
  if isinstance(node, ast.Num):  # <number>
/workspace/.pyenv_mirror/user/current/lib/python3.12/site-packages/joblib/_utils.py:39: DeprecationWarning: Attribute n is deprecated and will be removed in Python 

[(0,
  '0.009*"said" + 0.009*"cancer" + 0.005*"also" + 0.005*"carbon" + 0.004*"bjp" + 0.004*"year" + 0.003*"people" + 0.003*"say" + 0.003*"forest" + 0.003*"two"'),
 (1,
  '0.009*"galaxy" + 0.005*"year" + 0.005*"also" + 0.005*"stock" + 0.004*"company" + 0.004*"like" + 0.004*"million" + 0.004*"google" + 0.004*"said" + 0.004*"new"'),
 (2,
  '0.009*"court" + 0.008*"trump" + 0.006*"khan" + 0.005*"case" + 0.004*"said" + 0.004*"willis" + 0.004*"measles" + 0.004*"justice" + 0.003*"wade" + 0.003*"election"'),
 (3,
  '0.006*"said" + 0.004*"also" + 0.003*"like" + 0.003*"game" + 0.003*"year" + 0.003*"source" + 0.003*"new" + 0.003*"good" + 0.003*"ireland" + 0.003*"first"'),
 (4,
  '0.007*"season" + 0.006*"game" + 0.005*"also" + 0.005*"coach" + 0.004*"said" + 0.004*"year" + 0.004*"team" + 0.004*"million" + 0.004*"last" + 0.004*"nfl"')]

In [3]:
from datetime import date, timedelta

# Get today's date
today = date.today()

# Create a timedelta object with one day difference
yesterday = today - timedelta(days=1)

# Print yesterday's date

blob_name = f"news-{yesterday}.parquet"
blob_name

'news-2024-03-24.parquet'